#Importing Functions

In [1]:
!pip install tensorflow_model_optimization

In [2]:
import sys
sys.path.append('/content/drive/My Drive/Colab/MS Project/qkeras-master/qkeras-master')
import tensorflow_model_optimization as tfmot
from qkeras import *


In [3]:
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from scipy import ndimage
import numpy as np
from matplotlib import pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D,Conv2D
from keras.layers.convolutional import MaxPooling1D,MaxPooling2D
from keras.layers import BatchNormalization,ReLU,GlobalAveragePooling1D,MaxPooling1D,LSTM,TimeDistributed,GlobalAveragePooling2D
from keras.utils import to_categorical
from keras.models import save_model, load_model
from sklearn.preprocessing import StandardScaler
from keras.utils import to_categorical
from tensorflow.keras.utils import  plot_model
from keras.models import Model,save_model,load_model
from keras.layers import Input
from keras.layers.merge import concatenate
import numpy as np
from six.moves import zip
from tensorflow.keras import callbacks
import tensorflow.keras.backend as K
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import *
from tensorflow.keras.utils import to_categorical
from qkeras.utils import load_qmodel
from qkeras.estimate import print_qstats
from datetime import datetime

from packaging import version

import os
import tempfile
import os
import tensorflow as tf
from tensorflow import expand_dims
from tensorflow import keras
%load_ext tensorboard

In [4]:
!pip install -U tensorboard_plugin_profile

Requirement already up-to-date: tensorboard_plugin_profile in /usr/local/lib/python3.6/dist-packages (2.3.0)


#Loading Dataset

In [5]:
def load_file(filepath):
  dataframe=read_csv(filepath,header=None,delim_whitespace=True)
  return dataframe.values

In [6]:
def load_group(filenames,prefix=''):
  loaded=list()
  for name in filenames:
    data=load_file(prefix+name)
    loaded.append(data)
  loaded=dstack(loaded)
  return loaded

In [7]:
def load_dataset_group(group,prefix=''):
  filepath=prefix+group+'/Inertial Signals/'
  filenames=list()
  filenames+=['total_acc_x_'+group+'.txt','total_acc_y_'+group+'.txt','total_acc_z_'+group+'.txt']
  filenames+=['body_acc_x_'+group+'.txt','body_acc_y_'+group+'.txt','body_acc_z_'+group+'.txt']
  filenames+=['body_gyro_x_'+group+'.txt','body_gyro_y_'+group+'.txt','body_gyro_z_'+group+'.txt']
  X=load_group(filenames,filepath)
  y=load_file(prefix+group+'/y_'+group+'.txt')
  return (X,y)


In [8]:
def load_dataset(prefix=''):
  trainX,trainy=load_dataset_group('train',prefix+'/content/drive/My Drive/Colab/MS Project/UCI HAR Dataset/UCI HAR Dataset/')
  print(trainX.shape,trainy.shape)
  testX,testy=load_dataset_group('test',prefix+'/content/drive/My Drive/Colab/MS Project/UCI HAR Dataset/UCI HAR Dataset/')
  print(testX.shape,testy.shape)
  trainy=trainy-1
  testy=testy-1
  trainy=to_categorical(trainy)
  testy=to_categorical(testy)
  print(trainX.shape, trainy.shape, testX.shape, testy.shape)
  return trainX, trainy, testX, testy

In [9]:
def scale_data(trainX, testX):
	# remove overlap
	cut = int(trainX.shape[1] / 2)
	longX = trainX[:, -cut:, :]
	# flatten windows
	longX = longX.reshape((longX.shape[0] * longX.shape[1], longX.shape[2]))
	# flatten train and test
	flatTrainX = trainX.reshape((trainX.shape[0] * trainX.shape[1], trainX.shape[2]))
	flatTestX = testX.reshape((testX.shape[0] * testX.shape[1], testX.shape[2]))
	# standardize

	s = StandardScaler()
		# fit on training data
	s.fit(longX)
		# apply to training and test data
	longX = s.transform(longX)
	flatTrainX = s.transform(flatTrainX)
	flatTestX = s.transform(flatTestX)
	# reshape
	flatTrainX = flatTrainX.reshape((trainX.shape))
	flatTestX = flatTestX.reshape((testX.shape))
	return flatTrainX, flatTestX

##Function for model size

In [10]:
def get_zipped_model_size(file):
  # Returns size of gzipped model, in bytes.
  import os
  import zipfile

  zipped_file = file+'.zip'
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)

  return os.path.getsize(zipped_file)

In [11]:
optimizer=Adam(lr=0.0001)
with_bn=1
THRESHOLD=0.1

class LearningRateAdjuster(callbacks.Callback):
  def __init__(self):
    self.learning_rate_factor = 1.0
    pass

  def on_epoch_end(self, epochs, logs):
    max_variance = -1

    for layer in self.model.layers:
      if layer.__class__.__name__ in [
          "BatchNormalization",
          "QBatchNormalization"
      ]:
        variance = np.max(layer.get_weights()[-1])
        if variance > max_variance:
          max_variance = variance

    if max_variance > 32 and self.learning_rate_factor < 100:
      learning_rate = K.get_value(self.model.optimizer.learning_rate)
      self.learning_rate_factor /= 2.0
      print("***** max_variance is {} / lr is {} *****".format(
          max_variance, learning_rate))
      K.eval(K.update(
          self.model.optimizer.learning_rate, learning_rate / 2.0
      ))

lra = LearningRateAdjuster()

##1. CNN MODEL


In [12]:
trainX,trainy,testX,testy=load_dataset()
verbose,epochs,batch_size=1,500,32
n_timesteps,n_features,n_outputs=trainX.shape[1],trainX.shape[2],trainy.shape[1]
print('n step: ', n_timesteps)
print('features: ',n_features)
trainX,testX=scale_data(trainX,testX)
print(trainX.shape)

(7352, 128, 9) (7352, 1)
(2947, 128, 9) (2947, 1)
(7352, 128, 9) (7352, 6) (2947, 128, 9) (2947, 6)
n step:  128
features:  9
(7352, 128, 9)


In [13]:

'''

inputs=keras.Input(shape=(n_timesteps,n_features))


conv_1=tf.keras.layers.Conv1D(filters=64,kernel_size=5,strides=2,activation='relu')(inputs)
maxpool_1=tf.keras.layers.MaxPooling1D(pool_size=2,strides=2)(conv_1)

conv_2=tf.keras.layers.Conv1D(filters=128,kernel_size=3,strides=1,activation='relu')(maxpool_1)
maxpool_2=tf.keras.layers.MaxPooling1D(pool_size=2,strides=1)(conv_2)

conv_3=tf.keras.layers.Conv1D(filters=32,kernel_size=3,strides=1,activation='relu')(maxpool_2)
avg_pooling=tf.keras.layers.GlobalAveragePooling1D()(conv_3)
batch_norm=tf.keras.layers.BatchNormalization()(avg_pooling)

output=tf.keras.layers.Dense(n_outputs,activation='softmax')(batch_norm)
model=tf.keras.Model(inputs=inputs,outputs=output)
'''

x=x_in=Input(shape=(n_timesteps,n_features),name='input')
x=QActivation("quantized_relu_po2(4,1)",name="acti")(x)
x=QConv1D(filters=64, kernel_size=3,
    strides=1,
    kernel_quantizer=quantized_po2(4, 1),
    bias_quantizer=quantized_bits(4,1) ,
    name="conv1d_0_m")(x)
x = QActivation("quantized_relu(3,1)", name="act0_m")(x)
x=tf.keras.layers.MaxPooling1D(pool_size=2,strides=2)(x)
x=QConv1D(filters=128, kernel_size=3,
    strides=1,
    kernel_quantizer=quantized_po2(4, 1),
    bias_quantizer=quantized_bits(4,1),
    name="conv1d_1_m")(x)
x = QActivation("quantized_relu(3,1)", name="act1_m")(x)
x=tf.keras.layers.MaxPooling1D(pool_size=2,strides=2)(x)
x=QConv1D(filters=32, kernel_size=3,
    strides=1,
    kernel_quantizer=quantized_po2(4, 1),
    bias_quantizer=quantized_bits(4,2,2),
    name="conv1d_2_m")(x)
x = QActivation("quantized_relu(3,1)", name="act2_m")(x)
x=tf.keras.layers.GlobalAveragePooling1D()(x)
if with_bn:
  x=QBatchNormalization(
      gamma_quantizer=quantized_relu_po2(4,8),
      variance_quantizer=quantized_relu_po2(6),
      beta_quantizer=quantized_po2(4, 4),
      gamma_range=8,
      beta_range=4,
      name="bn1")(x)
x = QActivation("quantized_relu(3,1)", name="act3_m")(x)
x = Flatten()(x)
x = QDense(
    n_outputs,
    kernel_quantizer=quantized_ulaw(4, 0, 1),
    bias_quantizer=quantized_bits(4, 0, 1),
    name="dense")(
        x)
x = Activation("softmax", name="softmax")(x)
model = Model(inputs=[x_in], outputs=[x])

/content/drive/My Drive/Colab/MS Project/qkeras-master/qkeras-master/qkeras/qnormalization.py:81: UserWarning: gamma_range is deprecated in QBatchNormalization layer.
  warnings.warn('gamma_range is deprecated in QBatchNormalization layer.')
/content/drive/My Drive/Colab/MS Project/qkeras-master/qkeras-master/qkeras/qnormalization.py:84: UserWarning: beta_range is deprecated in QBatchNormalization layer.
  warnings.warn('beta_range is deprecated in QBatchNormalization layer.')


In [14]:
plot_model(model, show_shapes=True, to_file='/content/drive/My Drive/Colab/MS Project/My Model/CNN_Model.png')
model.compile(
    loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
history = model.fit(
      trainX, trainy, batch_size=batch_size,
      epochs=epochs, initial_epoch=1, verbose=1,
      validation_split=0.1,
      callbacks=[]) #lra])

outputs = []
output_names = []

for layer in model.layers:
  if layer.__class__.__name__ in [
        "QActivation", "QBatchNormalization", "Activation", "QDense",
        "QConv1D", "QDepthwiseConv2D"
  ]:
    output_names.append(layer.name)
    outputs.append(layer.output)

model_debug = Model(inputs=[x_in], outputs=outputs)

outputs = model_debug.predict(trainX)

print("{:30} {: 8.4f} {: 8.4f}".format(
      "input", np.min(trainX), np.max(trainX)))

for n, p in zip(output_names, outputs):
  print("{:30} {: 8.4f} {: 8.4f}".format(n, np.min(p), np.max(p)), end="")
  layer = model.get_layer(n)
  for i, weights in enumerate(layer.get_weights()):
    if layer.get_quantizers()[i]:
      weights = K.eval(layer.get_quantizers()[i](K.constant(weights)))
    print(" ({: 8.4f} {: 8.4f})".format(np.min(weights), np.max(weights)),
            end="")
  print("")

score = model.evaluate(testX, testy, verbose=False)
print("Test score:", score[0])
print("Test accuracy:", score[1])
FILE='/content/drive/My Drive/Colab/MS Project/qkeras-master/qkeras-master/My Qkeras Model/model.h5'
model.save(FILE)
print_qstats(model)


Epoch 2/500
207/207 [==============================] - 3s 16ms/step - loss: 1.1617 - accuracy: 0.5115 - val_loss: 1.1874 - val_accuracy: 0.4783
Epoch 3/500
207/207 [==============================] - 2s 12ms/step - loss: 0.5241 - accuracy: 0.8176 - val_loss: 0.4568 - val_accuracy: 0.8628
Epoch 4/500
207/207 [==============================] - 2s 11ms/step - loss: 0.3748 - accuracy: 0.8729 - val_loss: 0.4450 - val_accuracy: 0.8247
Epoch 5/500
207/207 [==============================] - 2s 12ms/step - loss: 0.3203 - accuracy: 0.8956 - val_loss: 0.2501 - val_accuracy: 0.9239
Epoch 6/500
207/207 [==============================] - 2s 12ms/step - loss: 0.2937 - accuracy: 0.8986 - val_loss: 0.2386 - val_accuracy: 0.9130
Epoch 7/500
207/207 [==============================] - 2s 11ms/step - loss: 0.2780 - accuracy: 0.9036 - val_loss: 0.2172 - val_accuracy: 0.9348
Epoch 8/500
207/207 [==============================] - 2s 12ms/step - loss: 0.2637 - accuracy: 0.9070 - val_loss: 0.1745 - val_accuracy:

In [15]:
logs = "logs/" + datetime.now().strftime("%Y%m%d-%H%M%S")

tboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logs,
                                                 histogram_freq = 1,
                                                 )
model.summary()


Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 128, 9)]          0         
_________________________________________________________________
acti (QActivation)           (None, 128, 9)            0         
_________________________________________________________________
conv1d_0_m (QConv1D)         (None, 126, 64)           1792      
_________________________________________________________________
act0_m (QActivation)         (None, 126, 64)           0         
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 63, 64)            0         
_________________________________________________________________
conv1d_1_m (QConv1D)         (None, 61, 128)           24704     
_________________________________________________________________
act1_m (QActivation)         (None, 61, 128)          

In [16]:

model_size=get_zipped_model_size(FILE)
print(" Size of qkeras cnn model: {:.2f}KB".format(model_size/1000))

 Size of qkeras cnn model: 443.68KB


#Post-Training Quantization

In [17]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
tflite_model = converter.convert()

tflite_file = '/content/drive/My Drive/Colab/MS Project/qkeras-master/qkeras-master/My Qkeras Model/qkeras_quantized_cnn_model.tflite'

with open(tflite_file, 'wb') as f:
  f.write(tflite_model)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /tmp/tmpg6fpdjng/assets


In [18]:
tflite_model_size=get_zipped_model_size(tflite_file)
print(" Size of quantized cnn model: {:.2f}KB".format(tflite_model_size/1000))

 Size of quantized cnn model: 33.22KB


In [19]:
def evaluate_model(interpreter,testX,testy):
  '''
# Get input and output tensors.
  input_details = interpreter.get_input_details()
  output_details = interpreter.get_output_details()
  print(input_details)
  print(output_details)
  print("== Input details ==")
  print("shape:", input_details[0]['shape'])
  print("type:", input_details[0]['dtype'])
  print("type:", input_details[0]['index'])
  print("\n== Output details ==")
  print("shape:", output_details[0]['shape'])
  print("type:", output_details[0]['dtype'])
  print("type:", output_details[0]['index'])



  input_shape = input_details[0]['index']
  output_shape = output_details[0]['index']

  # Run predictions on every image in the "test" dataset.
  prediction= []
  print(testX.shape)
  for i,test_data in enumerate(testX):
    # Pre-processing: add batch dimension and convert to float32 to match with
    # the model's input data format.

    test_data = np.expand_dims(test_data, axis=0).astype(np.float32)
  
    interpreter.set_tensor(input_shape, test_data)

    # Run inference.
    interpreter.invoke()

    # Post-processing: remove batch dimension and find the digit with highest
    # probability.
    output = interpreter.get_tensor(output_shape)
    digit = np.argmax(output,axis=1)
    prediction.append(digit)

  # Compare prediction results with ground truth labels to calculate accuracy.
  
  print('\n')
  prediction=np.array(prediction)
  print(prediction.shape)
  print(testy.shape)
  labels=np.argmax(testy,axis=-1)
  accuracy_count=0
  for i in range(len(prediction)):
    
    if(prediction[i]==labels[i]):
      accuracy_count+=1
  
  accuracy=(accuracy_count/len(prediction))*100

  return accuracy
  '''
  #Get input and output tensors.
  input_details = interpreter.get_input_details()
  output_details = interpreter.get_output_details()
  print(input_details)
  print(output_details)
  print("== Input details ==")
  print("shape:", input_details[0]['shape'])
  print("type:", input_details[0]['dtype'])
  print("type:", input_details[0]['index'])
  print("\n== Output details ==")
  print("shape:", output_details[0]['shape'])
  print("type:", output_details[0]['dtype'])
  print("type:", output_details[0]['index'])

  input_shape = input_details[0]['index']
  output_shape = output_details[0]['index']

  total_seen=0
  correct=0

  for data,label in zip(testX,testy):
    data=np.expand_dims(data, axis=0).astype(np.float32)
    total_seen+=1
    interpreter.set_tensor(input_shape,data)
    interpreter.invoke()
    predictions=interpreter.get_tensor(output_shape)
    if np.argmax(predictions)==np.argmax(label):
      correct+=1
    if total_seen%1000==0:
      print("Accuracy after {:d} data : {:f}".format(total_seen,float(correct)/float(total_seen)))

  return float(correct)/float(total_seen)

In [20]:

interpreter=tf.lite.Interpreter(tflite_file)
interpreter.allocate_tensors()

tflite_accuracy = evaluate_model(interpreter,testX,testy)

print('Base TFLite test_accuracy:', tflite_accuracy)

[{'name': 'input', 'index': 0, 'shape': array([  1, 128,   9], dtype=int32), 'shape_signature': array([ -1, 128,   9], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
[{'name': 'Identity', 'index': 174, 'shape': array([1, 6], dtype=int32), 'shape_signature': array([-1,  6], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
== Input details ==
shape: [  1 128   9]
type: <class 'numpy.float32'>
type: 0

== Output details ==
shape: [1 6]
type: <class 'numpy.float32'>
type: 174
Accuracy after 1000 data : 0.909000
Accuracy after 2000 data : 0.908000
Base TFLite test_accuracy: 0.9205972175093315
